In [189]:
import qsharp

In [208]:
%%qsharp


open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Diagnostics;

operation getGraphLength():Int{
    return 4;
}

operation getGraphWeigths():Double[][]{
    return [[0.,100.,0.,0.], [100.,0.,0.,0.], [0.,0.,0.,0.], [0.,0.,0.,0.]];
}

operation completeCycle(gamma: Double, beta: Double, p: Int): Double {
    return meanCostInverse(QAOA(gamma, beta, p));
}

operation QAOA(gamma:Double, beta:Double, p:Int):Int[]{
    
    //input register
    use register = Qubit[getGraphLength()];
    
    //prepare qubits by creating uniform superposition
    ApplyToEach(H, register);

    //repeat cost and mixer steps p times
    //I think that we might need a new gamma and beta for each p
    for i in 1..p{
        costLayer(register, gamma);
        mixerLayer(register, beta);
    }

    //Read the results as int array
    mutable res = new Int[Length(register)];
    for i in 0..Length(register)-1{
        set res w/= i <- (M(register[i]) == One ? 1 | 0);
    }

    //return the results
    return res;
}

operation mixerLayer(register:Qubit[], beta:Double):Unit{
    ApplyToEach(Rx(2.*beta, _), register); //the '2*beta' is from the qiskit vid https://www.youtube.com/watch?v=YpLzSQPrgSc
}
    
operation costLayer(register:Qubit[], gamma:Double):Unit{
    ApplyToEach(Rz(gamma, _), register); //should be weighted I think but idk how
    let weights = getGraphWeigths();
    for i in 0..Length(register)-1{
         for j in i+1..Length(register)-1{
             Rzz(gamma*weights[i][j], register[i], register[j]);
         }
     }
}
   
operation meanCostInverse(register: Int[]) : Double {
    mutable cost = 0.;//IntAsDouble(Length(register));
    let weights = getGraphWeigths();
    for i in 0 .. Length(register) - 1 {
        for j in i..Length(register)-1{
            set cost += weights[i][j];
        }
    }
    for i in 0 .. Length(register) - 1 {
        for j in i..Length(register)-1{
            let xi = IntAsDouble(register[1]);
            let xj = IntAsDouble(register[j]);
            set cost -= weights[i][j]*(1.-xi*xj)/2.; //from https://www.youtube.com/watch?v=aFVnWq3RHYU&t=126s
        }
    }
    return cost / IntAsDouble(Length(register));
}


D:\snippet_.qs(39,19): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.


In [209]:
for i in range(10):
    print(QAOA.simulate(gamma=1, beta=1, p=10))

[1, 0, 1, 0]
[0, 1, 1, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 1]
[1, 1, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[1, 1, 0, 0]


In [210]:
import numpy as np
from scipy.optimize import minimize

In [211]:
def obj_func(params):
    myres=completeCycle.simulate(gamma=params[0], beta=params[1], p=1);
    print('res= ', myres)
    print('params= ', params)


In [212]:
def getMostFrequent(gamma, beta):
    myDict = {}
    for i in range(1000):
        res = QAOA.simulate(gamma=gamma, beta=beta, p=1)
        res=tuple(res)
        if res in myDict.keys():
            myDict.update({res:myDict.get(res)+1})
        else:
            myDict.update({res:0})
    return np.array(max(myDict))

In [213]:
print(getMostFrequent(0.,0.))

[1 1 1 1]


In [214]:
def obj_func2 (params):
    myregister=getMostFrequent(params[0], params[1])
    print('params= ', params)
    print('register= ', myregister)
    myRes=meanCostInverse.simulate(register=myregister)
    print('cost= ', myRes)
    return myRes

In [215]:
print(obj_func2([2.,2.]))

params=  [2.0, 2.0]
register=  [1 1 1 1]
cost=  25.0
25.0


In [216]:
res = minimize(obj_func2,[0.0, 0.0], method='COBYLA')
print('all= ', res)

params=  [0. 0.]
register=  [1 1 1 1]
cost=  25.0
params=  [1. 0.]
register=  [1 1 1 1]
cost=  25.0
params=  [0. 1.]
register=  [1 1 1 1]
cost=  25.0
params=  [0.125 0.   ]
register=  [1 1 1 1]
cost=  25.0
params=  [0.    0.125]
register=  [1 1 1 1]
cost=  25.0
params=  [0.015625 0.      ]
register=  [1 1 1 1]
cost=  25.0
params=  [0.       0.015625]
register=  [1 1 1 1]
cost=  25.0
params=  [0.00195312 0.        ]
register=  [1 1 1 1]
cost=  25.0
params=  [0.         0.00195312]
register=  [1 1 1 1]
cost=  25.0
params=  [0.00024414 0.        ]
register=  [1 1 1 1]
cost=  25.0
params=  [0.         0.00024414]
register=  [1 1 1 1]
cost=  25.0
params=  [5.e-05 0.e+00]
register=  [1 1 1 1]
cost=  25.0
params=  [0.e+00 5.e-05]
register=  [1 1 1 1]
cost=  25.0
all=       fun: 25.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 13
  status: 1
 success: True
       x: array([0., 0.])


In [204]:
def obj_func3 (params):
    myregister=QAOA.simulate(gamma=params[0], beta=params[1], p=1)
    #print('params= ', params)
    #print('register= ', myregister)
    myRes=meanCostInverse.simulate(register=myregister)
    print('cost= ', myRes)
    return myRes

In [205]:
print(obj_func3([2.,2.]))

cost=  0.875
0.875


In [206]:
for i in range(5):
    res = minimize(obj_func3,[i, i+3.0], method='COBYLA')
    print('i= ',i,' i+3= ', i+3)
    print('all= ', res)

cost=  1.0
cost=  1.0
cost=  0.875
cost=  0.875
cost=  1.0
cost=  0.875
cost=  0.875
cost=  1.0
cost=  0.875
cost=  1.0
cost=  1.0
cost=  1.0
cost=  0.875
cost=  1.0
cost=  0.875
cost=  0.875
cost=  1.0
cost=  1.0
cost=  1.0
cost=  0.875
cost=  0.875
cost=  0.875
cost=  1.0
cost=  1.0
i=  0  i+3=  3
all=       fun: 1.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 24
  status: 1
 success: True
       x: array([-1.e-04,  4.e+00])
cost=  0.875
cost=  1.0
cost=  0.875
cost=  0.875
cost=  0.875
cost=  1.0
cost=  0.875
cost=  1.0
cost=  0.875
cost=  0.875
cost=  0.875
cost=  1.0
cost=  0.875
cost=  1.0
cost=  0.875
cost=  0.875
cost=  0.875
cost=  0.875
cost=  1.0
cost=  0.875
i=  1  i+3=  4
all=       fun: 0.875
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 20
  status: 1
 success: True
       x: array([1.0001, 4.    ])
cost=  0.875
cost=  0.875
cost=  1.0
cost=  1.0
cost=  1.0
cost=  1.0
cost=  0.875
cost=  1.0
cost=  1.0
cost=  1.0
